In [10]:
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser.text import SentenceSplitter
from llama_index.schema import TextNode
import chromadb
import os

In [11]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Text chunking configuration


In [12]:
chunk_size = 500  # Fixed chunk size in characters
text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=0)

In [13]:
chroma_db_path = "./chroma_db"
chroma_db = chromadb.PersistentClient(path=chroma_db_path)
chroma_collection = chroma_db.get_or_create_collection("aws_documentation")

In [14]:
#for ingestion pipeline we can set up a s3 bucket. user inputs their files -> sent to s3 bucket. We retrieve file from s3 and put it through the pipeline.
documents = (
    SimpleDirectoryReader(input_dir="C:/Users/Adel/Desktop/aws-documentation-main/aws-documentation-main/documents/alexa-for-business-administration-guide/doc_source", recursive=False, required_exts=[".md" , ".pdf"]).load_data())


In [16]:
text_chunks = []
for doc in documents:
    cur_text_chunks = text_splitter.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)

In [22]:
embeddings = []
metadatas = []
documents = []
ids = []
for i, chunk in enumerate(text_chunks):
    node = TextNode(text=chunk)
    node_embedding = embed_model.get_text_embedding(node.get_content())
    embeddings.append(node_embedding)
    if not node.metadata:
        node.metadata = {"AWS Documentation" : chunk}  # Set a default metadata
        metadatas.append(node.metadata)
    documents.append(chunk)
    ids.append(str(i)) 

In [23]:
chroma_collection.add(
    embeddings=embeddings,
    metadatas=metadatas,
    documents=documents,
    ids=ids
)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [34]:
query_str = "Can you tell me about alexa for business"


In [35]:
from llama_index.llms import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf" #using 13b for the time being until i can confirm that everything is working 100%
##you can change to 70b by just replacing the linkg

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '5120', 'llama.block_count': '40', 'llama.feed_forward_length': '13824', 'llama.attention.head_count': '40', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '2', 'llama.attention.head_count_kv': '40', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}


In [36]:
query_embedding = embed_model.get_query_embedding(query_str)

In [37]:
from llama_index.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [41]:
query_result = chroma_collection.query(vector_store_query)
print(query_result.nodes[0].get_content())

ValueError: Expected embeddings to be a list, got VectorStoreQuery(query_embedding=[0.022433791309595108, -0.09785578399896622, -0.011647995561361313, 0.006709470879286528, 0.011408106423914433, -0.013800844550132751, 0.04528016597032547, 0.06373386830091476, -0.02525698021054268, 0.0414503738284111, 0.009026887826621532, 0.07963839918375015, -0.014284857548773289, -0.02764747478067875, 0.02016776241362095, 0.03039521723985672, 0.06011497229337692, -0.10150458663702011, -0.0508599616587162, 0.004058341030031443, 0.07232792675495148, 0.023175304755568504, -0.09448248893022537, 0.016420023515820503, -0.05738189443945885, -0.030371155589818954, 0.009154521860182285, -0.02135523036122322, -0.057115212082862854, -0.08295891433954239, 0.019572829827666283, -0.037262093275785446, 0.06178784370422363, 0.08227599412202835, 0.05767480656504631, 0.06485921144485474, 0.020707594230771065, 0.02573552541434765, 0.03777938708662987, 0.026480207219719887, 0.006713313050568104, -0.03499256446957588, -0

In [40]:
from llama_index.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

NameError: name 'query_result' is not defined

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******